In [1]:
%matplotlib qt

import os
import pathlib

import matplotlib.pyplot as plt

import numpy as np

# ExaFMM-T
import exafmm.laplace as laplace

# PyExaFMM
from fmm import Fmm
from fmm.kernel import laplace_p2p_serial
from fmm.surface import scale_surface

# Plotting parameters
plt.rc('font', family='serif', serif='Times')
plt.rc('text', usetex=True)
plt.rc('xtick', labelsize=8)
plt.rc('ytick', labelsize=8)
plt.rc('axes', labelsize=8)

# Dimensions for column plots
# width = 3.487 # half page plot width
width = 4.328 # full page plot width
height = width / 1.618 # golden ratio

HERE = pathlib.Path(os.getcwd())
FIGURE_SAVEPATH = os.path.abspath(HERE.parent.parent / 'article/figures')

## Generate Test Data
Uncomment to generate test data

In [49]:
# ! rm test.hdf5 && fmm generate-test-data -c test && fmm compute-operators -c test
# ! rm C2E2.hdf5 && fmm generate-test-data -c C2E2 && fmm compute-operators -c C2E2
# ! rm C3E3.hdf5 && fmm generate-test-data -c C3E3 && fmm compute-operators -c C3E3
# ! rm C4E4.hdf5 && fmm generate-test-data -c C4E4 && fmm compute-operators -c C4E4
# ! rm C5E5.hdf5 && fmm generate-test-data -c C5E5 && fmm compute-operators -c C5E5
# ! rm C6E6.hdf5 && fmm generate-test-data -c C6E6 && fmm compute-operators -c C6E6
# ! rm C7E7.hdf5 && fmm generate-test-data -c C7E7 && fmm compute-operators -c C7E7

In [50]:
# Compile test experiment to cache numba functions
e = Fmm('test')
e.run()
laplace_p2p_serial(e.sources, e.targets, e.source_densities)

## Compare Analyticity of Expansions from PyExaFMM and ExaFMM-T

How does discretisation effect the quality of the expansions? We check the analyticity of multipole expansions by computing the approximated potential at points distributed on the surface of a sphere surrounding the equivalent surface, in the far-field. As the radius of this sphere is increased, we can demonstrate the accuracy of the multipole expansion away from the equivalent surface. We repeat a similar experiment for the local expansions.

In [51]:
def sphere_data(npoints, r, c):
    """
    Generate npoints randomly distributed on the
        surface of a sphere with radius r, and center c
    """
    np.random.seed(0)
    
    phi = np.random.rand(npoints)*2*np.pi
    costheta = (np.random.rand(npoints)-0.5)*2

    theta = np.arccos(costheta)

    x = r * np.sin(theta) * np.cos(phi)
    y = r * np.sin(theta) * np.sin(phi)
    z = r * np.cos(theta)

    sources = np.vstack((x, y, z))

    return sources.T + c


In [52]:
# test = sphere_data(1000, 10, np.array([10, 10, 0]))

# ax = plt.axes(projection='3d')
# ax.scatter3D(test[:, 0], test[:, 1], test[:, 2], c='k', s=1)
# plt.show()

In [53]:
# Discretisation order of check surface
Cvec = [2, 3, 4, 5, 6, 7]

# Discretisation order of equivalent surface
Evec = [2, 3, 4, 5, 6, 7]

# Load experimental data into PyExaFMM
pyfmmvec = [Fmm(f'C{C}E{C}') for C in Cvec]

In [54]:
# Load experimental data into ExaFMM-T
exafmmvec = []
exafmmtreevec = []

for e in pyfmmvec:
    # create a list of source instances
    sources = laplace.init_sources(e.sources, e.source_densities)

    # create a list of target instances
    targets = laplace.init_targets(e.targets)
    
    # Expansion order
    p = e.config['order_equivalent']
    fmm = laplace.LaplaceFmm(p=p, ncrit=e.config['max_points'], filename=f'C{p}E{p}.dat')
    exafmmvec.append(fmm)
    
    tree = laplace.setup(sources, targets, fmm)
    exafmmtreevec.append(tree)

In [55]:
# Evaluate PyExaFMM experiments
for e in pyfmmvec:
    e.run()

In [56]:
# Evaluate ExaFMM-T experiments
ex_target_potentials = []
for tree, fmm in list(zip(exafmmtreevec, exafmmvec)):
    ex_target_potentials.append(laplace.evaluate(tree, fmm))

### Check convergence of multipole expansion of root node

In [57]:
# Create surfaces
equivalent_surfaces = [
    e.equivalent_surface for e in pyfmmvec
]

upward_equivalent_surfaces = [scale_surface(
    surf=equivalent_surfaces[i], radius=pyfmmvec[i].r0, 
    level=0, center=pyfmmvec[i].x0, alpha=pyfmmvec[i].alpha_outer
) for i in range(len(pyfmmvec))]

# Extract multipole expansion at root node
pyfmm_multipole_expansions = [
    e.multipole_expansions[e.key_to_index[0]:e.key_to_index[0]+e.nequivalent_points]
    for e in pyfmmvec
]

exafmm_multipole_expansions = [
    np.array(tree.nodes[0].up_equiv) for tree in exafmmtreevec
]

In [58]:
# Create spheres centered on the root node with radii such that
# the root node's multipole expansion is valid
npoints = 1000
r0 = pyfmmvec[0].r0
x0 = pyfmmvec[0].x0

# Start of far-field
r = r0*3
c = x0
rvec = np.linspace(r, 10*r, 10)
spherevec = [sphere_data(npoints, r, c) for r in rvec]


# Store results for each experiment
pyfmm_results = [[] for i in range(len(pyfmmvec))]
exafmm_results = [[] for i in range(len(pyfmmvec))]
direct_results = [[] for i in range(len(pyfmmvec))]


# Compare expansion results with direct computation at spheres of increasing radius
# surrounding the upward equivalent surface of the root node.
for e_idx in range(len(pyfmmvec)):
    for i, sphere in enumerate(spherevec):

        pyfmm_results[e_idx].append(
            laplace_p2p_serial(
                sources=upward_equivalent_surfaces[e_idx],
                targets=sphere,
                source_densities=pyfmm_multipole_expansions[e_idx]
            )
        )

        exafmm_results[e_idx].append(
            laplace_p2p_serial(
                sources=upward_equivalent_surfaces[e_idx],
                targets=sphere,
                source_densities=exafmm_multipole_expansions[e_idx]
            )
        )

        direct_results[e_idx].append(
            laplace_p2p_serial(
                sources=pyfmmvec[e_idx].sources,
                targets=sphere,
                source_densities=pyfmmvec[e_idx].source_densities
            )
        )
        
pyfmm_results = np.array(pyfmm_results)
exafmm_results = np.array(exafmm_results)
direct_results = np.array(direct_results)

In [59]:
# Compute relative errors of multipole expansions of PyFMM and ExaFMM-T
pyfmm_relative_errors = [[] for i in range(len(pyfmmvec))]
exafmm_relative_errors = [[] for i in range(len(pyfmmvec))]

for e_idx in range(len(pyfmmvec)):
    for sph_idx in range(len(spherevec)):
        direct = direct_results[e_idx][sph_idx]
        pyfmm_estimate = pyfmm_results[e_idx][sph_idx]
        exafmm_estimate = exafmm_results[e_idx][sph_idx]
        
        pyfmm_err = abs(direct-pyfmm_estimate)/direct
        exafmm_err = abs(direct-exafmm_estimate)/direct

        pyfmm_relative_errors[e_idx].append(np.mean(pyfmm_err))
        exafmm_relative_errors[e_idx].append(np.mean(exafmm_err))
        
pyfmm_relative_errors = np.array(pyfmm_relative_errors)
exafmm_relative_errors = np.array(exafmm_relative_errors)

In [60]:
fig, ax = plt.subplots()
fig.subplots_adjust(left=.15, bottom=.16, right=.99, top=.97)

lines = ['solid', 'dashed', 'dashdot', 'dotted', 'solid', 'dashed']
markers = ['.', 'o', 'v', '+', 'D', 's']

for e_idx in range(len(pyfmmvec)):
    ax.semilogy(
        rvec, 
        pyfmm_relative_errors[e_idx], 
        marker=markers[e_idx], 
        linestyle=lines[e_idx],
        linewidth='0.4',
        markersize=2
    )

# Label with number of coefficients rather than expansion order
legend_labels = [6*(order-1)**2 + 2 for order in Evec]
plt.legend(legend_labels)
plt.xlabel('$R_s$')
plt.ylabel('Relative Error $\epsilon_{rel}$')
fig.set_size_inches(width, height)
fp = FIGURE_SAVEPATH  + '/pyexafmm_multipole_convergence.pdf'
plt.savefig(fp)

In [61]:
fig, ax = plt.subplots()
fig.subplots_adjust(left=.15, bottom=.16, right=.99, top=.97)

lines = ['solid', 'dashed', 'dashdot', 'dotted', 'solid', 'dashed']
markers = ['.', 'o', 'v', '+', 'D', 's']

for e_idx in range(len(pyfmmvec)):
    ax.semilogy(
        rvec, 
        exafmm_relative_errors[e_idx], 
        marker=markers[e_idx], 
        linestyle=lines[e_idx],
        linewidth='0.4',
        markersize=2
    )

# Label with number of coefficients
legend_labels = [ 6*(order-1)**2 + 2 for order in Evec]
plt.legend(legend_labels)
plt.xlabel('$R_s$')
plt.ylabel('Relative Error $\epsilon_{rel}$')
fig.set_size_inches(width, height)
fp = FIGURE_SAVEPATH  + '/exafmm_multipole_convergence.pdf'
plt.savefig(fp)
plt.show()

### Check convergence of local expansion of a given leaf node

### Convergence of Potentials

In [45]:
# Calculate potential directly
direct_potential = laplace_p2p_serial(
    sources=pyfmmvec[0].sources,
    targets=pyfmmvec[0].targets, 
    source_densities=pyfmmvec[0].source_densities
)

In [62]:
# Store results for each experiment
pyfmm_potential_errors = []
exafmm_potential_errors = []


# Compare expansion results with direct computation at spheres of increasing radius
# surrounding the upward equivalent surface of the root node.
for e_idx in range(len(pyfmmvec)):
    pyfmm_estimate = pyfmmvec[e_idx].target_potentials
    pyfmm_error = np.mean(abs(direct_potential-pyfmm_estimate)/direct_potential)
    pyfmm_potential_errors.append(pyfmm_error)
    
    exafmm_estimate = ex_target_potentials[e_idx]
    exafmm_error = np.mean(abs(direct_potential-exafmm_estimate[:,0])/direct_potential)
    exafmm_potential_errors.append(exafmm_error)

In [64]:
fig, ax = plt.subplots()
fig.subplots_adjust(left=.15, bottom=.16, right=.99, top=.97)

ax.semilogy(
    Evec, 
    exafmm_potential_errors, 
    marker='o', 
    linestyle='--',
    linewidth='0.4',
    markersize=2
)


ax.semilogy(
    Evec, 
    pyfmm_potential_errors, 
    marker='x', 
    linestyle=':',
    linewidth='0.4',
    markersize=2
)


# Label with number of coefficients
legend_labels = ['ExaFMM', 'PyExaFMM']

plt.legend(legend_labels)
plt.xlabel('Order of Expansions')
plt.ylabel('Relative Error $\epsilon_{rel}$')
fig.set_size_inches(width, height)
fp = FIGURE_SAVEPATH  + '/potential_convergence.pdf'
plt.savefig(fp)
plt.show()